# Question 1
Extend the tic-tac-toe game covered in Week 6 Lecture 1 from a 3x3 grid to a 4x4 grid. 
Bonus points if you can generalize this for any N x N grid.  (40 points)

In [1]:
import time
import numpy as np
from math import inf as infinity
class Game:
    def __init__(self,n):
        self.initialize_game(n)
    def initialize_game(self,n):
        self.board = [[' ' for i in range(n)] for j in range(n)]
        self.size = n
        self.board = np.array(self.board)
       # Player X always plays first
        self.player_turn = 'X'
        self.hasher = {}
                
    def get_emptycells(self):
      cells = []
      for i in range (self.size):
        for j in range (self.size):          
          if(self.board[i][j] == ' '):
            cells.append([i,j])
      return cells       

    def draw_board(self):
        hline =  '-'*(self.size*4+2)+'\n'
        vbar = ' | '
        out_buff = hline 
        for row in range(self.size):
          out_buff += vbar 
          for col in range(self.size):
              out_buff += str(self.board[row][col])
              out_buff += vbar 
          out_buff += '\n' + hline
        print(out_buff) 


    def hashboard(self):
      hashseq = ' '
      for i in range(self.size):
        for j in range(self.size):
          if self.board[i][j] == 'X':
            hashseq = hashseq+'1'
          elif self.board[i][j] == 'O':
            hashseq = hashseq+'2'
          else:
            hashseq = hashseq+'0'  
      return hashseq
      
    def is_valid(self, px, py):
        if px < 0 or px > (self.size-1) or py < 0 or py > (self.size-1):
            return False
        elif self.board[px][py] != ' ':
            return False
        else:
            return True     

      #Checks if the game has ended and returns the winner in each case
    def evaluate(self):
        # Vertical win
        for col in range(self.size):
          col_uniq = np.unique(self.board[:,col])  
          if (col_uniq.shape[0]==1 and col_uniq[0]!= ' '):
            return self.board[0][col]
        # Horizontal win
        for row in range(self.size):
          row_uniq = np.unique(self.board[row])
          if (row_uniq.shape[0]== 1 and row_uniq[0] != ' '  ):
            return self.board[row][0]

        # Main diagonal win
        diag = self.board.diagonal()
        if (np.unique(diag).shape[0]==1 and diag[0]!= ' '):
          return diag[0]
        
       # Second diagonal win
        antidiag = np.fliplr(self.board).diagonal()
        if (np.unique(antidiag).shape[0]==1 and antidiag[0]!= ' '):
          return antidiag[0]

      # Is whole board full?
        for i in range(0, self.size):
          for j in range(0, self.size):
              # There's an empty field, we continue the game
              if (self.board[i][j] == ' '):
                return None

      # It's a tie!
        return ' '
    
    def minmax(self,depth,alpha,beta,isMax):
      result = self.evaluate()
      if result == 'X':
          return (-1, 0, 0)
      elif result == 'O':
          return (1, 0, 0)
      elif result == ' ':
          return (0, 0, 0)
          ##
      if (isMax):
        best = -infinity
        px = None
        py = None
        emptycells = self.get_emptycells()
        for cell in emptycells:
          i,j = cell
          self.board[i][j] = 'O'
          temphash = self.hashboard()
          #hash board pattern
          if (temphash in self.hasher.keys()):
            (m, min_i, min_j) = (self.hasher[temphash],i,j) 
            #dont compute if already calculated
          else:
            (m, min_i, min_j) = self.minmax(depth+1,alpha,beta,False)
            self.hasher[temphash]  = m
          #self.draw_board()
          # Fixing the best value if needed
          if m > best:
            best = m
            px,py = i,j
          # Setting back the field , undo
          self.board[i][j] = ' '
          if best > alpha:
            alpha = best  
          if alpha>=beta:
            #print('hi debug')# pruning
            return(best,px,py)
        return (best, px, py)
                           
      else:  
        best = infinity
        qx = None
        qy = None 
        emptycells = self.get_emptycells()
        for cell in emptycells:
          i,j = cell
          self.board[i][j] = 'X'
          temphash = self.hashboard()#hash boad pattern
          if (temphash in self.hasher.keys()):
            (m, max_i, max_j) = (self.hasher[temphash],i,j) #dont compute if already calculated
          else:
            (m, max_i, max_j) = self.minmax(depth+1,alpha,beta,True)
            self.hasher[temphash] = m

          if m < best:
            best = m
            qx = i
            qy = j
          self.board[i][j] = ' '
          if best<beta:
            beta = best
          if beta <= alpha:
            #print('hi debug')
            return (best,qx,qy)# pruning
        return (best, qx, qy)
                
    def play(self):
      while True:
          self.draw_board()
          self.result = self.evaluate()

        # Printing the appropriate message if the game has ended
          if self.result != None:
              if self.result == 'X':
                  print('The winner is X!')
              elif self.result == 'O':
                  print('The winner is O!')
              elif self.result == ' ':
                  print("It's a tie!")
              #print('hi')
              self.initialize_game(3)
              return

        # If it's player's turn
          if self.player_turn == 'X':

              while True:

                  start = time.time()
                  loc = int(input("Enter cell number\n"))
                  end = time.time()
                  print('Human Evaluation time: {}s'.format(round(end - start, 7)))
                  (px, py) = (int((loc-1)/self.size),(loc-1)%self.size)  
                  (qx, qy) = (px, py)
                  if self.is_valid(px, py):
                      self.board[px][py] = 'X'
                      self.player_turn = 'O'
                      break
                  else:
                      print('The move is not valid! Try again.')

          else:
              start = time.time()
              (m, px, py) = self.minmax(0,-infinity,infinity,True)
              end = time.time()
              print('Agent Evaluation time: {}s'.format(round(end - start, 7)))
              self.board[px][py] = 'O'
              self.player_turn = 'X'        
 



 

In [2]:
n=4
a = Game(n)#n=grid size
a.play()

------------------
 |   |   |   |   | 
------------------
 |   |   |   |   | 
------------------
 |   |   |   |   | 
------------------
 |   |   |   |   | 
------------------

Enter cell number
9
Human Evaluation time: 3.4282799s
------------------
 |   |   |   |   | 
------------------
 |   |   |   |   | 
------------------
 | X |   |   |   | 
------------------
 |   |   |   |   | 
------------------

Agent Evaluation time: 25.7358491s
------------------
 | O |   |   |   | 
------------------
 |   |   |   |   | 
------------------
 | X |   |   |   | 
------------------
 |   |   |   |   | 
------------------

Enter cell number
2
Human Evaluation time: 34.1786592s
------------------
 | O | X |   |   | 
------------------
 |   |   |   |   | 
------------------
 | X |   |   |   | 
------------------
 |   |   |   |   | 
------------------

Agent Evaluation time: 0.0004513s
------------------
 | O | X | O |   | 
------------------
 |   |   |   |   | 
------------------
 | X |   |   |   | 
-

# Question 2
You suddenly have a lot of assignments to submit in the next 24 hours. You need to sort them based on their properties, so you can solve them one by one. Each assignment has the following properties - no. of questions, average time required to solve a question, and total marks, and . Implement all three of the sorting algorithms you have learned to sort them, to maximize the marks you can get, within the time constraint. Also print the time taken by each.
 
Note that you may not have the time to solve all the assignments, and sorting in decreasing order of marks may not be the optimal solution in every case. (30 points) 

In [15]:
import random
# tabulate
from tabulate import tabulate
class Assignment:
  '''An assignment has the following 3 properties:
  q: no. of questions
  m: total  marks
  t: average time required to solve each question'''
  def __init__(self):
    self.q = random.randrange(1,15)
    self.m = random.randrange(1,50)
    self.t = random.randrange(1,60)

def AssignmentGenerator(n):
  '''
  Takes parameter - n
  Return list of n assignment objects
  '''
  assignments = []
  for i in range(n):
    assignments.append(Assignment())
  return assignments


# Driver Program
n = int(input("Enter the number of assignments : "))
assgn_lst = AssignmentGenerator(n)

assgn_wghts = [i.m/(i.t*i.q) for i in assgn_lst]

# print all data in a table format
tablelist = []
num = 1

for i in assgn_lst:
  templist = []
  templist.append(('Assignment'+str(num)))
  templist.append(i.q*i.t)
  templist.append(i.m)
  templist.append(i.q)
  templist.append(i.t)
  templist.append(i.m/(i.q*i.t))
  tablelist.append(templist)
  num = num+1

print(tabulate(tablelist, headers=['Assignment','Total Time to Complete','Total Marks','No of Questions','Avg Time for each Qstn','Marks to Time ratio']))
print('\n')
print('To  maximize the marks please follow below order')
exectimelist= {}

'''
Implementation of Bubble Sort
'''


def bubsort(ptablelist,passgn_wghts ):
   global exectimelist
   start = time.time()
   for i in range(len(passgn_wghts)-1,0,-1):
    change = 0
    for j in range(i):
      if passgn_wghts[j]<passgn_wghts[j+1]:
        temp,temp1 = passgn_wghts[j],ptablelist[j]
        passgn_wghts[j],ptablelist[j]= passgn_wghts[j+1],ptablelist[j+1]
        passgn_wghts[j+1],ptablelist[j+1] = temp,temp1
        change = 1
    if(change==0):
     break
   end = time.time()
   exectimelist['Bubble Sort']=(end - start)
   print('Bubble Sort Execution time: {}s'.format(round(end - start, 7)))  
   print('Bubble Sort Result')   
   print(tabulate(ptablelist, headers=['Assignment', 'Total Time to Complete','Total Marks','No of Questions','Avg Time for each Qstn','Marks to Time ratio']))

bubsort(tablelist,assgn_wghts)
'''
Implementation of Insertion Sort
'''
def insertionsort(ptablelist,passgn_wghts ):
  global exectimelist
  start = time.time()
  for i in range(1,len(passgn_wghts)):
    key1,key2 = passgn_wghts[i],ptablelist[i]
    j = i-1
    while(j>=0 and key1<passgn_wghts[j]):
      passgn_wghts[j+1],ptablelist[j+1] = passgn_wghts[j],ptablelist[j]
      j=j-1
    passgn_wghts[j+1],ptablelist[j+1] = key1,key2
    #passgn_wghts,ptablelist = passgn_wghts[-1::-1],ptablelist[-1::-1] 
  passgn_wghts=passgn_wghts[-1::-1]
  ptablelist=ptablelist[-1::-1] 
  end = time.time()
  exectimelist['Insertion Sort']=(end - start)
  print('\n\nInsertion  Sort Execution time: {}s'.format(round(end - start, 7)))
  print('Insertion Sort Result')  
  print(tabulate(ptablelist, headers=['Assignment', 'Total Time to Complete','Total Marks','No of Questions','Avg Time for each Qstn','Marks to Time ratio']))
insertionsort(tablelist,assgn_wghts)

'''
Implementation of merge Sort
'''
def merge(leftw,rightw,leftassgn,rightassgn):

    resultw = []
    resultassgn = []
    while len(leftw) != 0 and len(rightw) != 0:
        if leftw[0] > rightw[0]:
            resultw.append(leftw[0])
            resultassgn.append(leftassgn[0])
            leftw.remove(leftw[0])
            leftassgn.remove(leftassgn[0])
        else:
            resultw.append(rightw[0])
            resultassgn.append(rightassgn[0])
            rightw.remove(rightw[0])
            rightassgn.remove(rightassgn[0])
    if len(leftw) == 0:
        resultw = resultw + rightw
        resultassgn = resultassgn + rightassgn
    else:
        resultw = resultw + leftw
        resultassgn = resultassgn + leftassgn
    return list(resultw),list(resultassgn) 

def merge_sort(ptablelist,passgn_wghts):
    if len(passgn_wghts) <= 1:
        return passgn_wghts,ptablelist
    middle = len(passgn_wghts) // 2
    leftw = passgn_wghts[:middle]
    rightw = passgn_wghts[middle:]
    leftassgn = ptablelist[:middle]
    rightassgn = ptablelist[middle:]

    leftw,leftassgn = merge_sort(leftassgn,leftw)
    rightw,rightassgn = merge_sort(rightassgn,rightw)
    return merge(leftw, rightw,leftassgn,rightassgn)
start = time.time()
ww,ll=merge_sort(tablelist,assgn_wghts)   
end = time.time()
exectimelist['Merge Sort']=(end - start)
print('\n\nMerge Sort Execution time: {}s'.format(round(end - start, 7))) 
print('MergeSort Result') 
print(tabulate(ll, headers=['Assignment', 'Total Time to Complete','Total Marks','No of Questions','Avg Time for each Qstn','Marks to Time ratio']))

#print(tabulate([exectimelist[i] for i in exectimelist.keys()], headers=exectimelist.keys()))
print('All results of sorting methods are printed at once to check any conflicting orderring sequence.')



Enter the number of assignments : 9
Assignment      Total Time to Complete    Total Marks    No of Questions    Avg Time for each Qstn    Marks to Time ratio
------------  ------------------------  -------------  -----------------  ------------------------  ---------------------
Assignment1                        408              4                 12                        34             0.00980392
Assignment2                        410             44                 10                        41             0.107317
Assignment3                        261             39                  9                        29             0.149425
Assignment4                        120             40                 12                        10             0.333333
Assignment5                        171             33                  9                        19             0.192982
Assignment6                          9             36                  1                         9             4
Assig

In [16]:
print(exectimelist)

{'Bubble Sort': 1.9550323486328125e-05, 'Insertion Sort': 1.4543533325195312e-05, 'Merge Sort': 0.00011014938354492188}


# Question 3
Given a directed graph representing all the courses offered by a department, where an edge from A to B indicates that course A is a sufficient prerequisite for eligibility to course B, find a path from any course to the Brain Computer Interfacing course. Use DFS to check if a path exists, and print the path found. Then print the shortest path using BFS. (20 points)

In [19]:
class Graph:
  '''
  A graph uses a dict(gdict) {vertex : [edges]}
  '''

  def __init__(self, v_list =None):
    self.gdict = {}
    if v_list is not None:
      for vertex in v_list:
        self.addVertex(vertex)
  
  def addVertex(self, vertex):
    self.gdict[vertex] = []
  
  def addEdge(self, start, end):
    if start not in self.gdict.keys():
      print("Start vertex not found : ", start)
    if type(end) == list:
      for vertex in end:
        self.addEdge(start, vertex)
    elif end not in self.gdict.keys():
      print("End vertex not found : ", end)
    else:
      self.gdict[start].append(end)
  
  def getVertices(self):
    return self.gdict.keys()
  
  def getNeighbours(self, vertex):
    if vertex in self.gdict.keys():
      return self.gdict[vertex]
    else:
      print("Vertex not found.")
    #Depth first search  
  def dfs(self,initialnode):
    destination = "Brain Computer Interfacing"
    visited = {}
    qeuelist = []
    pathdict = {}
    pathdict[initialnode] = initialnode
    qeuelist.append(initialnode) 
    for i in self.gdict:
      visited[i] = False
    while len(qeuelist)!=0:
      templist = [] 
      firstnode  = qeuelist[0]
      if (visited[firstnode]== False):
        visited[firstnode] = True
        rootnodelist = self.getNeighbours(firstnode)
        for i in rootnodelist:
          templist.append(i)
          pathdict[i] = pathdict[firstnode]+'->'+i
          if (i==destination):
            return pathdict[i]
        qeuelist = templist+qeuelist[1:]
      else:  
        qeuelist.remove(firstnode)
    print('You cannot reach '+destination+'with given branch - '+initialnode)

   # Breadth first search 
  def bfs(self,initialnode):
    destination = "Brain Computer Interfacing"
    visited = {}
    qeuelist = []
    pathdict = {}
    pathdict[initialnode] = initialnode
    qeuelist.append(initialnode) 
    for i in self.gdict:
      visited[i] = False
    while len(qeuelist)!=0:
      #print(qeuelist)
      templist = [] 
      firstnode  = qeuelist[0]
      if (visited[firstnode]== False):
        visited[firstnode] = True
        rootnodelist = self.getNeighbours(firstnode)
        for i in rootnodelist:
          templist.append(i)
          pathdict[i] = pathdict[firstnode]+'->'+i
          if (i==destination):
            return pathdict[i]
        qeuelist = qeuelist[1:]+templist
      else:  
        qeuelist.remove(firstnode)
    print('You cannot reach '+destination+'with given branch - '+initialnode)          

       
  
courses = ["Biology", "Physics", "Chemistry", "Computer Science", "Cognitive Psychology", "Psychology", "Cognitive Neuroscience", "Anatomy", "Neurobiology", "Neuroscience", 
           "Logic and Cognitive Science", "Social Psychology", "Game Design", "Lego Robotics", "Robotics", "Mechanics", "Neural Interfacing", "Cognitive Science", 
           "Computational Tools in Cognitive Science", "Philosophy of Mind", "Anthropology", "Linguistics", "Machine Learning", "Artificial Intelligence", 
           "Natural Language Processing", "Embodied Cognition", "Electronics",  "Haptic Systems", "Signal Processing", "Embedded Systems", "Brain Computer Interfacing"]
  
course_graph = Graph(courses)

course_graph.addEdge("Biology", ["Anatomy", "Neurobiology", "Neuroscience","Cognitive Science"])
course_graph.addEdge("Physics", ["Mechanics", "Electronics", "Cognitive Science", "Game Design"])
course_graph.addEdge("Chemistry", ["Neurobiology", "Neuroscience", "Cognitive Science"])
course_graph.addEdge("Linguistics", ["Anthropology","Artificial Intelligence", "Natural Language Processing", "Cognitive Science"])
course_graph.addEdge("Computer Science", ["Cognitive Science", "Computational Tools in Cognitive Science", "Artificial Intelligence", "Game Design"])
course_graph.addEdge("Psychology", ["Cognitive Psychology", "Social Psychology", "Anthropology", "Cognitive Science"])
course_graph.addEdge("Mechanics", ["Lego Robotics", "Robotics", "Electronics", "Cognitive Science", "Embedded Systems"])
course_graph.addEdge("Electronics", ["Mechanics", "Embedded Systems", "Lego Robotics", "Robotics", "Signal Processing", "Haptic Systems", "Cognitive Science"])
course_graph.addEdge("Cognitive Science", ["Cognitive Psychology", "Neurobiology", "Cognitive Neuroscience", "Logic and Cognitive Science","Cognitive Science", "Linguistics", "Computational Tools in Cognitive Science", "Philosophy of Mind", "Anthropology", "Embodied Cognition"])
course_graph.addEdge("Cognitive Psychology", ["Social Psychology", "Anthropology", "Artificial Intelligence"])
course_graph.addEdge("Philosophy of Mind", ["Anthropology", "Artificial Intelligence"])
course_graph.addEdge("Neurobiology", ["Neuroscience", "Neural Interfacing"])
course_graph.addEdge("Anatomy", ["Neurobiology", "Haptic Systems", "Embodied Cognition"])
course_graph.addEdge("Computational Tools in Cognitive Science", ["Artificial Intelligence", "Embedded Systems", "Game Design"])
course_graph.addEdge("Logic and Cognitive Science", ["Artificial Intelligence", "Signal Processing", "Game Design"])
course_graph.addEdge("Artificial Intelligence", ["Machine Learning", "Natural Language Processing","Game Design", "Embodied Cognition"])
course_graph.addEdge("Machine Learning", ["Lego Robotics", "Robotics", "Natural Language Processing"])
course_graph.addEdge("Game Design", ["Lego Robotics", "Haptic Systems"])
course_graph.addEdge("Lego Robotics", ["Robotics", "Signal Processing", "Embedded Systems"])
course_graph.addEdge("Robotics", ["Haptic Systems", "Signal Processing", "Embedded Systems"])
course_graph.addEdge("Signal Processing", ["Neural Interfacing", "Embedded Systems"])
course_graph.addEdge("Embedded Systems", ["Robotics"])
course_graph.addEdge("Neuroscience", ["Neural Interfacing",  "Brain Computer Interfacing"])
course_graph.addEdge("Haptic Systems", ["Signal Processing"])
course_graph.addEdge("Neural Interfacing", ["Haptic Systems", "Signal Processing", "Embedded Systems", "Brain Computer Interfacing"])

###
# DFS Traversal
# Traverse the course_graph using DFS, and print the path (if found)
'''
Defined Member Function dfs

'''

# BFS Traversal
# Traverse the course_graph using BFS, and return the shortest path

'''
Defined Member Function bfs

''' 

# Driver Program
for i in range(len(courses)):
  print(i+1,courses[i])
source_node_num = int(input("Enter the course number to be used as the starting subject to reach Brain Computer Interfacing : \n"))
source_node = courses[source_node_num-1]
bfs_path = course_graph.bfs(source_node)
dfs_path=course_graph.dfs(source_node)
if(bfs_path):
  x = len(bfs_path.split('->'))
  print('BFS path of Prerequisite courses-'+str(x-1)+' :'+bfs_path)

if(dfs_path):
  x = len(dfs_path.split('->'))
  print('DFS path of Prerequisite courses-'+str(x-1)+' :'+dfs_path)


1 Biology
2 Physics
3 Chemistry
4 Computer Science
5 Cognitive Psychology
6 Psychology
7 Cognitive Neuroscience
8 Anatomy
9 Neurobiology
10 Neuroscience
11 Logic and Cognitive Science
12 Social Psychology
13 Game Design
14 Lego Robotics
15 Robotics
16 Mechanics
17 Neural Interfacing
18 Cognitive Science
19 Computational Tools in Cognitive Science
20 Philosophy of Mind
21 Anthropology
22 Linguistics
23 Machine Learning
24 Artificial Intelligence
25 Natural Language Processing
26 Embodied Cognition
27 Electronics
28 Haptic Systems
29 Signal Processing
30 Embedded Systems
31 Brain Computer Interfacing
Enter the course number to be used as the starting subject to reach Brain Computer Interfacing : 
16
BFS path of Prerequisite courses-5 :Mechanics->Electronics->Cognitive Science->Neurobiology->Neural Interfacing->Brain Computer Interfacing
DFS path of Prerequisite courses-6 :Mechanics->Lego Robotics->Robotics->Haptic Systems->Signal Processing->Neural Interfacing->Brain Computer Interfacing

# Question 4
Implement a simple Queue using linked lists with the following functions - insert, delete, display. Create an instance of Queue and present a menu to the user to perform operations on the queue. (10 points)


In [21]:
class Node: 
  '''
  A node contains the data it holds and a pointer to the next node
  '''
  def __init__(self, data): 
    self.data = data 
    self.next = None

class Queue:
  '''
  Create class Queue using a Linked List
  '''
  def __init__(self): 
    self.front = None
    self.rear = None
  def Insert(self,pdata):
    temp = Node(pdata)
    if(self.rear == None):
      self.rear = temp
      self.front = temp
      return 
    self.rear.next = temp
    self.rear = temp    
  def delete(self):
    if (self.front == None):
      return
    print(str(self.front.data)+' is deleted')  
    temp = self.front.next
    self.front = temp 
    if(self.front == None):
      self.rear= None

  def display(self):
    if(self.front==None):
      print('Queue is empty')
      return
    print('Queue elements are \n')
    qlist = []
    i= self.front
    qlist.append(i.data)
    while(i.next):
      i=i.next
      qlist.append(i.data)
    print(qlist)  
# Driver Program
q = Queue() 
print("Qeue operations\n1.Enqueeue\n2.Dequeue\n3.Disaply\n4.Quit")
while True:
  option=int(input('Enter option\n'))
  if option == 4:
    break
  elif option == 1:
    data = input('Enter data to add store\n')
    q.Insert(data)
  elif option ==2:
    q.delete()
  elif option == 3:
    q.display()  






    

Qeue operations
1.Enqueeue
2.Dequeue
3.Disaply
4.Quit
Enter option
3
Queue is empty
Enter option
1
Enter data to add store
5
Enter option
3
Queue elements are 

['5']
Enter option
1
Enter data to add store
10
Enter option
1
Enter data to add store
15
Enter option
3
Queue elements are 

['5', '10', '15']
Enter option
2
5 is deleted
Enter option
3
Queue elements are 

['10', '15']
Enter option
4
